### Create prompt dataset

In [2]:
%pip install -U sagemaker==2.151.0


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install transformers==4.27.2


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import json
  
# Opening JSON file
f1 = open('data-distributed-qanda-alt/all.jsonl')
Lines1 = f1.readlines()

questions, answers = [], []
for line in Lines1[:20000]:
    row = json.loads(line)
    for answer in row["human_answers"]:
        questions.append("Prompt: "+row["question"])
        answers.append("Response: "+answer)
    for answer in row["chatgpt_answers"]:
        questions.append("Prompt: "+row["question"])
        answers.append("Response: "+answer)

test_file = open("data-distributed-qanda-alt/test.jsonl","w")
for line in Lines1[20000:]:
    test_file.write(line)

test_file.close()


df = pd.DataFrame()
df["question"] = questions
df["answer"] = answers
df = df.sample(frac = 1)
df_train = df.iloc[:60000,:]
df_val = df.iloc[60000:,:]

df_train.to_csv("data-distributed-qanda-alt/train.csv", index=False)
df_val.to_csv("data-distributed-qanda-alt/val.csv", index=False)

### Upload data to S3

In [5]:
import sagemaker
from sagemaker.pytorch import PyTorch

In [6]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

region = sess.boto_region_name

bucket = sess.default_bucket()

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

sagemaker role arn: arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518
sagemaker bucket: sagemaker-us-east-1-079002598131
sagemaker session region: us-east-1


In [7]:
train_data_url = sess.upload_data(
    path="data-distributed-qanda-alt/train.csv",
    key_prefix="promptsds",
)

valid_data_url = sess.upload_data(
    path="data-distributed-qanda-alt/val.csv",
    key_prefix="promptsds",
)

In [8]:
print(f"training file path {train_data_url}")
print(f"validation file path {valid_data_url}")

training file path s3://sagemaker-us-east-1-079002598131/promptsds/train.csv
validation file path s3://sagemaker-us-east-1-079002598131/promptsds/val.csv


### Fine Tune FLAN T5 XXL (11b) on Seq2Seq

#### Store model files as checkpoints for easy deployment

In [9]:
from sagemaker.debugger import rule_configs
from sagemaker.debugger import DebuggerHookConfig, ProfilerRule

rules = [
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),   
]

hook_config = DebuggerHookConfig(
    hook_parameters={
        "save_interval": "10",  # number of steps
        "export_tensorboard": "true",
        "tensorboard_dir": "hook_tensorboard/",
    }
)

In [10]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(local_path="/opt/ml/output/profiler/", start_step=5, num_steps=10),
)

Framework profiling will be deprecated from tensorflow 2.12 and pytorch 2.0 in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
smp_options = {
    "enabled":True,
    "parameters": {                        # Required
        "pipeline_parallel_degree": 1,     # Required
        "ddp": True,
        "sharded_data_parallel_degree": 16, # Add this to activate sharded data parallelism
        "partitions":1,
        "bf16":True,
        "skip_tracing": True
    }
}

mpi_options = {
    "enabled" : True,                      # Required
    "processes_per_host" : 8               # Required
}

In [12]:
base_job_name="sft-flan-t5-11b"

checkpoint_s3_prefix = "flant5-checkpoints"
checkpoint_s3_uri = "s3://" + sess.default_bucket() + "/" + checkpoint_s3_prefix
print(checkpoint_s3_uri)

s3://sagemaker-us-east-1-079002598131/flant5-checkpoints


In [13]:
hyperparameters = {}

hyperparameters["model_name_or_path"] = "google/flan-t5-xxl"
hyperparameters["train_file"] = "/opt/ml/input/data/train/train.csv"
hyperparameters["validation_file"] = "/opt/ml/input/data/valid/val.csv"
hyperparameters["per_device_train_batch_size"]=8
hyperparameters["per_device_eval_batch_size"]=8
hyperparameters["block_size"]=2048
hyperparameters["model_dir"]="/opt/ml/model"
hyperparameters["num_train_epochs"]=1
hyperparameters["max_train_steps"]=100

In [14]:
# launch with smp

estimator = PyTorch(
    base_job_name=base_job_name,
    source_dir="src-distributed-qanda-alt/",
    entry_point="train.py",
    role=role,
    framework_version="1.13.1",
    py_version="py39", 
    instance_count=2,
    instance_type="ml.p4d.24xlarge", # Don't forget to change sharded_data_parallel_degree when you change this
    hyperparameters=hyperparameters,
    checkpoint_s3_uri=checkpoint_s3_uri,    
    disable_profiler=True,
#    keep_alive_period_in_seconds=600,    
    distribution={
        "smdistributed": {"modelparallel": smp_options},
        "mpi": mpi_options
    },
#    rules=rules,
#    debugger_hook_config=hook_config,
#    profiler_config=profiler_config,    
)

In [15]:
estimator.fit({
    "train": train_data_url,
    "valid": valid_data_url
}, wait=False)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: sft-flan-t5-11b-2023-05-03-02-31-52-537


In [16]:
training_job_name = estimator.latest_training_job.name
print("Training Job Name:  {}".format(training_job_name))

Training Job Name:  sft-flan-t5-11b-2023-05-03-02-31-52-537


In [17]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

In [18]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

In [19]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(
            bucket, training_job_name, region
        )
    )
)

In [20]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Checkpoint Data</a> After The Training Job Has Completed</b>'.format(
            bucket, checkpoint_s3_prefix, region
        )
    )
)

In [ ]:
%%time

estimator.latest_training_job.wait(logs=False)


2023-05-03 02:31:54 Starting - Starting the training job......
2023-05-03 02:32:33 Starting - Preparing the instances for training...................
2023-05-03 02:34:14 Downloading - Downloading input data.....
2023-05-03 02:34:44 Training - Downloading the training image...................
2023-05-03 02:36:24 Training - Training image download completed. Training in progress............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................